<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>


# Modeling groundwater flow in a conceptual catchment

<hr>
<small> For instructions on how to run an interactive iPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/release/README.md">https://github.com/landlab/tutorials/blob/release/README.md</a></small><br>
<small>For more Landlab tutorials, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

This tutorial demonstrates how the GroundwaterDupuitPercolator can be used to model groundwater flow and seepage (groundwater return flow). It is recommended to read the documentation for the component before starting this tutorial to be familiar with the mechanics of the model.

In this tutorial you will:
* Create a raster grid on which to run the model
* Simulate constant recharge and check that the component conserves mass
* Simulate recharge from storm events, check conservation of mass, and look at the outflow hydrograph

A bit of magic so that we can plot within this notebook.

In [ ]:
from __future__ import print_function
%matplotlib inline

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from landlab import RasterModelGrid, FIXED_VALUE_BOUNDARY, CLOSED_BOUNDARY, imshow_grid
from landlab.components import GroundwaterDupuitPercolator, FlowAccumulator
from landlab.components.uniform_precip import PrecipitationDistribution

## Create a RasterModelGrid

Here you will make the grid on which we will run the model. You will create three fields: topographic elevation, aquifer base elevation, and initial water table elevation

In [ ]:
boundaries = {'top': 'closed','bottom': 'closed','right':'closed','left':'closed'}
grid = RasterModelGrid((51, 51), spacing=10.0,bc=boundaries)
grid.status_at_node[1] = FIXED_VALUE_BOUNDARY 

elev = grid.add_zeros('node', 'topographic__elevation')
elev[:] = (0.001*grid.x_of_node**2 + 0.001*grid.y_of_node**2)+2

base = grid.add_zeros('node', 'aquifer_base__elevation')
base[:] = elev - 2

wt = grid.add_zeros('node', 'water_table__elevation')
wt[:] = elev

In [ ]:
plt.figure()
imshow_grid(grid,'topographic__elevation')

The grid is square with dimensions 500x500m. The surface elevation and aquifer base have the same concave parabolic shape, with thickness 2m between them. The aquifer is initially fully saturated (water table at the surface). Water is only allowed to exit the domain through a single node in the the lower left corner. All other boundaries are closed.  

## Simulate constant groundwater recharge

Now initialize the model components. In addition to the grid, the GroundwaterDupuitPercolator takes four optional arguments: hydraulic conductivity, porosity, recharge rate, and a regularization factor that smooths the transition between subsurface and surface flow as the water table approaches the ground surface. The greater the value, the smoother the transition.

You will also initialize a FlowAccumulator in order to use an included method to calculate the surface water discharge out of the domain. The runoff rate used by the FlowAccumulator is the surface water specific discharge from the groundwater model.

In [ ]:
K = 0.01 # hydraulic conductivity, (m/s)
R = 1e-7 # recharge rate, (m/s)
n = 0.2 # porosity, (-)
gdp = GroundwaterDupuitPercolator(grid, hydraulic_conductivity=K, porosity = n, recharge_rate=R,regularization_f=0.01)
fa = FlowAccumulator(grid, surface='topographic__elevation',
        flow_director='FlowDirectorSteepest', runoff_rate='surface_water__specific_discharge')

Next, run the model forward in time, and track the fluxes leaving the domain.

In [ ]:
N = 1000
dt = 1E2

recharge_flux = np.zeros(N)
gw_flux = np.zeros(N)
sw_flux = np.zeros(N)
storage = np.zeros(N)

for i in range(N):
    gdp.run_one_step(dt)
    
    fa.run_one_step()
    
    recharge_flux[i] = gdp.calc_recharge_flux_in()
    gw_flux[i] = gdp.calc_gw_flux_out()
    sw_flux[i] = gdp.calc_sw_flux_out()
    storage[i] = gdp.calc_total_storage()

Now visualize some results.

In [ ]:
plt.figure()
imshow_grid(grid,(wt-base)/(elev-base),cmap='Blues')

The above shows how saturated the aquifer is. Note that it is most saturated at the lowest area of the domain, nearest the outlet.

Now look at the mass balance by ploting cumulative fluxes. The cumulative recharge in should be equal to cumulative fluxes out (groundwater and surface water) plus the change in storage from the initial condition.

In [ ]:
t = np.arange(0,N*dt,dt)

plt.figure(figsize=(8,6))
plt.plot(t/3600,np.cumsum(gw_flux)*dt+np.cumsum(sw_flux)*dt+storage-storage[0],
                 'b-',linewidth=3, alpha=0.5,label='Total Fluxes + Storage' )
plt.plot(t/3600,np.cumsum(recharge_flux)*dt,'k:',label='recharge flux')
plt.plot(t/3600,np.cumsum(gw_flux)*dt,'b:',label='groundwater flux')
plt.plot(t/3600,np.cumsum(sw_flux)*dt,'g:',label='surface water flux')
plt.plot(t/3600,storage-storage[0], 'r:', label='storage')
plt.ylabel('Cumulative Volume $[m^3]$')
plt.xlabel('Time [h]')
plt.legend(frameon=False)
plt.show()

The thick blue line (cumulative fluxes plus storage) matches the black cumulative recharge flux line, which indicates that the model has conserved mass. Because the initial domain was fully saturated, the primary feature that shows up in this mass balance is the loss of that initial water. It will be easier to see what is going on here in the second example. 

## Simulate time-varying recharge

Lastly, simulate time-varying recharge, look at the mass balance, and the outflow hydrograph. This will use the same grid and groundwater model instance as above, taking the final condition of the previous model run as the new initial condition here. This time the adaptive timestep solver will be used to make sure the model remains stable.

First, we need a distribution of recharge events. We will use landlab's precipitation distribution tool to create a lists paired recharge events and intensities.

In [ ]:
#generate storm timeseries
T = 10*24*3600 #sec
Tr = 1*3600 #sec
Td = 24*3600 #sec
dt = 1e3 #sec
p = 1e-3 #m

precip = PrecipitationDistribution(mean_storm_duration=Tr, mean_interstorm_duration=Td, 
                                               mean_storm_depth=p, total_t=T, delta_t=dt)
durations = []
intensities = []
precip.seed_generator(seedval=1)
for (interval_duration, rainfall_rate_in_interval) in (
                precip.yield_storm_interstorm_duration_intensity(subdivide_interstorms=True)
):
   durations.append(interval_duration)
   intensities.append(rainfall_rate_in_interval)
N = len(durations)  

Next run the model forward with the run_with_adaptive_time_step_solver. This method is the same as run_one_step, except that it subdivides the provided timestep (event or inter-event duration in this case) in order to meet a Courant-type stability criterion. The argument courant_coefficient indicates how large the maximum allowed timestep is relative to the Courant limit. Values close to 0.1 are recommended for best results.

In [ ]:
recharge_flux = np.zeros(N)
gw_flux = np.zeros(N)
sw_flux = np.zeros(N)
storage = np.zeros(N)
num_substeps = np.zeros(N)

for i in range(N):
    gdp.recharge = intensities[i]*np.ones_like(gdp.recharge)
    gdp.run_with_adaptive_time_step_solver(durations[i], courant_coefficient=0.2)

    num_substeps[i] = gdp.number_of_substeps
    
    fa.run_one_step()

    recharge_flux[i] = gdp.calc_recharge_flux_in()
    gw_flux[i] = gdp.calc_gw_flux_out()
    sw_flux[i] = gdp.calc_sw_flux_out()
    storage[i] = gdp.calc_total_storage()

Again, visualize the mass balance:

In [ ]:
t = np.cumsum(durations)

plt.figure()
plt.plot(t/3600,np.cumsum(gw_flux*durations)+np.cumsum(sw_flux*durations)+storage-storage[0],
                 'b-',linewidth=3, alpha=0.5,label='Total Fluxes + Storage' )
plt.plot(t/3600,np.cumsum(recharge_flux*durations)-recharge_flux[0]*durations[0],'k:',label='recharge flux')
plt.plot(t/3600,np.cumsum(gw_flux*durations),'b:',label='groundwater flux')
plt.plot(t/3600,np.cumsum(sw_flux*durations),'g:',label='surface water flux')
plt.plot(t/3600,storage-storage[0], 'r:', label='storage')
plt.ylabel('Cumulative Volume $[m^3]$')
plt.xlabel('Time [h]')
plt.legend(frameon=False)
plt.show()

Visualize numer of substeps that the model took for stability:

In [ ]:
plt.figure()
plt.plot(num_substeps,'.')
plt.xlabel('Iteration')
plt.ylabel('Numer of Substeps')
plt.yticks([1,5,10,15,20])
plt.show()

In [ ]:
max(num_substeps)

The method has subdivided the timestep up to 18 times in order to meet the stability criterion. This is dependent on a number of factors, including the Courant coefficient, the hydraulic conductivity, and hydraulic gradient.

Now look at the timeseries of recharge in and groundwater and surface water leaving the domain at the open node:

In [ ]:
fig,ax = plt.subplots(figsize=(8,6))
ax.plot(t/(3600*24),sw_flux, label='Surface water flux')
ax.plot(t/(3600*24),gw_flux, label='Groundwater flux')
ax.set_ylim((0,0.04))
ax.set_ylabel('Flux out $[m^3/s]$')
ax.set_xlabel('Time [d]')
ax.legend(frameon=False,loc=7)
ax1 = ax.twinx()
ax1.plot(t/(3600*24),recharge_flux,'0.6')
ax1.set_ylim((1.2,0))
ax1.set_ylabel('Recharge flux in $[m^3/s]$')
plt.show()

The relationship between maximum flux that can be passed through the subsurface and the occurrence of groundwater seepage is clear from this figure.

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>